### Manual model run
This notebook does a single manual run of the model from the default parameters in ./inputs/parameters.yml and produces some basic plots, including the matrix scaling, which is not influenced by calibration to a significant extent.

In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch finalise-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH
from aust_covid.utils import add_image_to_doc
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.parameters import load_param_info

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'model_construction', 'Model construction', 'austcovid')
targets = get_targets(DummyTexDoc())
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()

In [ ]:
epi_model = build_model(app_doc)

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
outputs_fig = plot_single_run_outputs(epi_model, targets)
outputs_fig

In [ ]:
example_matrix_fig = plot_example_model_matrices(epi_model, parameters)
example_matrix_fig

In [ ]:
add_image_to_doc(outputs_fig, 'single_run_outputs', 'jpg', 'Outputs from single model run', app_doc, 'Outputs')
add_image_to_doc(example_matrix_fig, 'example_matrices', 'jpg', 'Dynamic mixing matrices', app_doc, 'Mixing')

In [ ]:
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex model_construction
    ! biber model_construction
    ! pdflatex model_construction
    ! pdflatex model_construction